In [1]:
import requests
import pandas as pd
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_project_data(project_id):
    API_URL = f"https://pad.crc.nd.edu/api/v2/projects/{project_id}/cards"
    try:
        # fetch_data_from_api
        r = requests.get(url=API_URL,verify=False)  # NOTE: Using verify=False due to a SSL issue, I need a valid certificate, then I will remove this parameter.
        r.raise_for_status() # Raise an exception if the status is not 200
        data = r.json()
        df = pd.json_normalize(data)
        return df
    except requests.exceptions.RequestException as e:
        print(e)
        print(f"Error accessing project data {project_id}: {r.status_code}")
        return None


In [2]:
project_id = 13
df = get_project_data(project_id)

In [3]:
df.columns.to_list()

['id',
 'sample_name',
 'test_name',
 'user_name',
 'date_of_creation',
 'raw_file_location',
 'processed_file_location',
 'processing_date',
 'camera_type_1',
 'notes',
 'sample_id',
 'quantity',
 'deleted',
 'issue',
 'project.id',
 'project.user_name',
 'project.project_name',
 'project.annotation',
 'project.test_name',
 'project.sample_names.sample_names',
 'project.neutral_filler',
 'project.qpc20',
 'project.qpc50',
 'project.qpc80',
 'project.qpc100',
 'project.notes']

In [4]:
## Counting unique values to identify categories in the 'sample_name' column

# Normalize case to lowercase and replace spaces with dashes in 'sample_name'
df['sample_name'] = df['sample_name'].str.lower().str.replace(' ', '-', regex=False)

# Counting unique values to identify categories
print(df['sample_name'].value_counts())

amoxicillin-rerun                    149
blank-pad-(no-reagents)              110
acetaminophen                         44
blank-pad-(stamped-with-reagents)     11
albendazole                            8
amoxicillin                            6
quinine                                3
                                       2
Name: sample_name, dtype: int64


In [5]:
# Sample random 3 cases for each unique sample_name
# If a group has less than 3 entries, it takes all from that group
sample_cases_per_sample_name = df.groupby('sample_name').apply(lambda x: x.sample(min(len(x), 20)))

print("\nRandom sample cases for unique quantities:")
#sample_cases_per_sample_name[['id', 'sample_name']]


Random sample cases for unique quantities:


In [6]:
import ipyplot
import matplotlib.pyplot as plt



label_column = 'sample_name'
data = sample_cases_per_sample_name[sample_cases_per_sample_name['sample_name'].isin(['blank-pad-(no-reagents)', 'blank-pad-(stamped-with-reagents)'])].copy()
# Define the base URL
base_url = "https://pad.crc.nd.edu"

# Concatenate to get the full image URLs
data['url'] = base_url + data['processed_file_location']

# The 'url' column now contains the full URLs to the images
images = data['url'].values
labels = data[label_column].values

# Labels for the tabs - unique quantities
labels_list_filtered = data[label_column].unique().tolist()

# Show with tabs
ipyplot.plot_class_tabs(
    images, labels, tabs_order=labels_list_filtered, img_width=150
)

plt.show() 

In [7]:
# Count unique 'sample_id's and total 'id's for each 'sample_name' and 'quantity' group
counts_per_group = df[df['sample_name'].isin(['blank-pad-(stamped-with-reagents)', 'blank-pad-(no-reagents)'])].groupby(['sample_name', 'quantity']).agg({
    'sample_id': 'nunique',  # Counts unique sample_ids
    'id': 'count'            # Counts total ids
})

print("Counts of 'sample_id's and 'id's by 'sample_name' and 'quantity' group:")
counts_per_group

Counts of 'sample_id's and 'id's by 'sample_name' and 'quantity' group:


,,sample_id,id
sample_name,quantity,,
blank-pad-(no-reagents),100,109,110
blank-pad-(stamped-with-reagents),100,11,11


In [8]:
sample_cases_per_sample_name

id  \
sample_name                                    
                                  325  31169   
                                  326  34728   
acetaminophen                     44   13748   
                                  43   13747   
                                  29   13718   
...                                      ...   
blank-pad-(stamped-with-reagents) 316  14129   
                                  309  14122   
quinine                           38   13742   
                                  40   13744   
                                  39   13743   

                                                             sample_name  \
sample_name                                                                
                                  325                                      
                                  326                                      
acetaminophen                     44                       acetaminophen   
                                  43                       acetaminophen   
                                  29                       acetaminophen   
...                                                                  ...   
blank-pad-(stamped-with-reagents) 316  blank-pad-(stamped-with-reagents)   
                                  309  blank-pad-(stamped-with-reagents)   
quinine                           38                             quinine   
                                  40                             quinine   
                                  39                             quinine   

                                                test_name  \
sample_name                                                 
                                  325  12LanePADKenya2015   
                                  326  12LanePADKenya2015   
acetaminophen                     44   12LanePADKenya2015   
                                  43   12LanePADKenya2015   
                                  29   12LanePADKenya2015   
...                                                   ...   
blank-pad-(stamped-with-reagents) 316  12LanePADKenya2015   
                                  309  12LanePADKenya2015   
quinine                           38   12LanePADKenya2015   
                                  40   12LanePADKenya2015   
                                  39   12LanePADKenya2015   

                                                      user_name  \
sample_name                                                       
                                  325               Chris Sweet   
                                  326             Tchounga Ange   
acetaminophen                     44   api-OUDPXFH17PEGKUW3FM4Z   
                                  43   api-OUDPXFH17PEGKUW3FM4Z   
                                  29                        api   
...                                                         ...   
blank-pad-(stamped-with-reagents) 316  api-06Q9CLAC8D69ECGB80AL   
                                  309  api-06Q9CLAC8D69ECGB80AL   
quinine                           38   api-OUDPXFH17PEGKUW3FM4Z   
                                  40   api-OUDPXFH17PEGKUW3FM4Z   
                                  39   api-OUDPXFH17PEGKUW3FM4Z   

                                          date_of_creation  \
sample_name                                                  
                                  325  2021-07-08T09:30:04   
                                  326  2021-09-01T07:25:03   
acetaminophen                     44   2019-01-22T13:14:37   
                                  43   2019-01-22T13:11:51   
                                  29   2019-01-11T14:46:43   
...                                                    ...   
blank-pad-(stamped-with-reagents) 316  2019-02-07T14:59:53   
                                  309  2019-02-07T14:59:44   
quinine                           38   2019-01-18T13:39:21   
                                  40   2019-01-18T14:25:06   
                  